In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import win32com.client
import getpass
import datetime
import pywintypes
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

<h2>ANN</h2>

In [ ]:
bills_clean = pd.read_pickle('C:\\SMU_v2\\bills_output_datefiltered.pkl')
CDM = pd.read_pickle("C:\\SMU_v2\\output.pkl")
        
clinical = CDM.drop(['dob','cause_of_death','death_age',\
                     'Date_for_DFS','Date_for_OS', 'Date_for_CSS',\
                     'Count_as_DFS', 'Count_as_OS','Count_as_CSS'], axis=1)

OHE = [i for i in clinical.columns if not (i in  ["NRIC", 'Age_@_Dx', 'size_precise', 'nodespos','dx_date'])]
x_clinical = pd.get_dummies(clinical,columns=OHE,dummy_na=True)

In [ ]:
prices_grouped = pd.read_pickle("C:\\SMU_v2\\price_timeperiod.pkl")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
prices_grouped_scaled = pd.DataFrame(mms.fit_transform(prices_grouped))

In [ ]:
#ann model
import tensorflow as tf
from tensorflow.keras import layers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
y = prices_grouped_scaled
X = x_clinical.drop(columns=["NRIC","dx_date"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(128, input_shape=(153,)))         # input layer
model.add(layers.Dense(128, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(64, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(64, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(64, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.sigmoid))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.sigmoid))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
model.add(layers.Dense(6, activation=tf.nn.leaky_relu))   # one output layer with 1 outputs

In [ ]:
# Compile the model with appropriate Loss function. metrics is something you can monitor (but model does not optimize metrc)
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mean_squared_error')

In [ ]:
# Run the stochastic gradient descent for specified epochs
epochs = 100
model.fit(X_train, y_train, epochs=epochs)

In [ ]:
test_loss = model.evaluate(X_test, y_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
predictions = pd.DataFrame(pred,columns=["6 months before", "6 months after", "1 year after", "2 years after", "5 years after", "10 years after"])

In [ ]:
predictions_scaled_reverse = pd.DataFrame(mms.inverse_transform(predictions))
y_test_scaled_reverse = pd.DataFrame(mms.inverse_transform(y_test))

In [ ]:
predictions_scaled_reverse.head()

In [ ]:
y_test_scaled_reverse.head()